### Quantile and Bucket Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
frame = pd.DataFrame({"data1":np.random.standard_normal(1000),
                     "data2":np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.453189,1.282881
1,-0.067475,-0.646802
2,-0.114406,-0.497899
3,0.311867,-0.416710
4,-1.071452,-0.501140


In [4]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head()

0    (-1.947, -0.362]
1     (-0.362, 1.223]
2     (-0.362, 1.223]
3     (-0.362, 1.223]
4    (-1.947, -0.362]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.538, -1.947] < (-1.947, -0.362] < (-0.362, 1.223] < (1.223, 2.808]]

In [5]:
def get_stats(group):
    return pd.DataFrame({"min":group.min(), 
                        "max": group.max(), 
                        "count":group.count(), 
                        "mean":group.mean()})

In [6]:
grouped = frame.groupby(quartiles)

In [7]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.538, -1.947] data1 -3.532006 -1.956655     32 -2.319901
                 data2 -2.451325  1.136086     32 -0.168747
(-1.947, -0.362] data1 -1.938414 -0.364652    333 -0.954492
                 data2 -2.655198  3.154389    333 -0.030731
(-0.362, 1.223]  data1 -0.361234  1.222645    526  0.356683
                 data2 -3.224476  3.620261    526 -0.046972
(1.223, 2.808]   data1  1.235387  2.808233    109  1.673185
                 data2 -2.839452  2.922840    109  0.083870

In [8]:
grouped.agg(["min", "max", "count", "mean", "std"])

data1                                         data2  \
                       min       max count      mean       std       min   
data1                                                                      
(-3.538, -1.947] -3.532006 -1.956655    32 -2.319901  0.333468 -2.451325   
(-1.947, -0.362] -1.938414 -0.364652   333 -0.954492  0.423191 -2.655198   
(-0.362, 1.223]  -0.361234  1.222645   526  0.356683  0.444771 -3.224476   
(1.223, 2.808]    1.235387  2.808233   109  1.673185  0.357742 -2.839452   

                                                      
                       max count      mean       std  
data1                                                 
(-3.538, -1.947]  1.136086    32 -0.168747  0.917643  
(-1.947, -0.362]  3.154389   333 -0.030731  1.042991  
(-0.362, 1.223]   3.620261   526 -0.046972  1.037118  
(1.223, 2.808]    2.922840   109  0.083870  1.026653

### Filling missing value with group specific value

In [ ]:
358